# Математическая модель для бота стилиста

В этой тетради будет краткое описание мат. модели для <a href=https://github.com/KeepCalmAndLoveML/HSEProject> моего ИВР</a>, а также прототип (совсем простой) на Питоне

### Базовая концепция

*Все значения могут быть разными в засимости от выбранного пола.*

Введём определения.
Пусть рекомендация $r, r \in [0; 5]$ - это число, которое определяет, насколько человеку подходит та или иная вещь (чем больше $r$, тем более вещь подходит).

Пусть $k = const$ - количество различных вещей, которые может порекомендовать бот.

Тогда результатом работы модели будет какое-то $R = \{r_1, r_2, ..., r_k\}$, где $r_i$ - рекомендация для i-той вещи.


Пусть $n = const$ - количество вводимых пользователем значений.
Пусть $V(i), i \le n, i \in \Bbb N$ - функция, которая возвращает i-тое введёное пользователем значение

Пусть параметр $P_i(x) = \{r_1, r_2, ..., r_k\}, i \le n, i \in \Bbb N$ - это функция, которая принимает i-тое по порядку значение, введёное пользователем, а возвращает вектор рекомендациий.
$P_i$ скорее всего будет задана, как кусочная функция

Пусть $W = \{w_1, w_2, ..., w_n\}$ - веса параметров.

Тогда $R$ можно посчитать по следующей формуле
$$ R = \frac{\sum_{i=0}^n P_i(V(i)) * w_i}{\sum_{i=0}^n w_i}$$

Таким образом, R - это средневзвешенное значение параметров и их весов

### Делаем математические рекомендации понятными для пользователя

Пусть $t = const$.

Считается, что i-тая вещь *подходит* человеку, если $R_i \ge t$

### Возможные улучшения

Одно из введёных пользователем значений будет "Тип фигуры". Это самое важное значение, и, возможно, стоит сделать значения вектора весов зависимым от этого значения. То есть, значение весов будет функцией, которая возвращает веса для остальных параметров в зависимости от введёного пользователем "Типа Фигуры".

# Прототип на Питоне

Далее будет простой прототип на питоне, который проиллюстрирует математическую модель, описанную выше.

In [19]:
k = 4
clothes = ['Длинное платье', 'Короткое платье', 'Юбка', 'Кожаная куртка']

n = 3
t = 2.5

def v(idx):
    #Рост / Вес
    if idx == 1:
        return 170 / 90
    
    #Рост
    if idx == 2:
        return 170 
    
    #Тип фигуры
    if idx == 2:
        return 1
    
    #Рост / Длина ног
    if idx == 3:
        return 170 / 70
    
#Рост / Вес
def p_1(x):
    default = [t] * k
    #Идеально
    if 175 / 50 <= x <= 175 / 65:
        return default
    
    #Слишком худая
    if x <= 175 / 50:
        return default
    
    #Слишком толстая
    if x >= 175 / 65:
        ans = list(default)
        ans[1] = 1 #Короткое платье ОЧЕНЬ не подходит толстым людям 
        return ans
    
#Рост
def p_2(x):
    #Для заданных вещей ничего не зависит от роста
    default = [t] * k
    return default

#Рост / Длина ног
def p_3(x):
    default = [t] * k
    #Короткие ноги
    if x < 175 / 75:
        ans = list(default)
        ans[1] = 2 #Короткое платье не очень подходит людям с короткими ногами
        return ans
    
    return default

w = [3, 1, 3]
p = [p_1, p_2, p_3]

In [38]:


r = []
for idx in range(k):
    up = 0
    for i in range(n):
        up += p[i](v(i + 1))[idx] * w[i]
    
    up /= sum(w)
    r.append(up)
    
print(r)

[2.5, 2.5, 2.5, 2.5]


In [39]:
for k, rk in enumerate(r):
    if rk >= t:
        print('Вам подходит {0}'.format(clothes[k]))

Вам подходит Длинное платье
Вам подходит Короткое платье
Вам подходит Юбка
Вам подходит Кожаная куртка


array([[ 7.5,  7.5,  7.5,  7.5],
       [ 2.5,  2.5,  2.5,  2.5],
       [ 7.5,  7.5,  7.5,  7.5]])